 **Set Up Spark Environment
install pyspark**

In [28]:
!pip install pyspark

In [29]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Nutrition Analysis") \
    .config("spark.jars.packages", "com.crealytics:spark-excel_2.12:0.13.5") \
    .getOrCreate()



**Starbucks drinks expanded dataset**

In [30]:
file_path = "/content/drive/MyDrive/Internship_projectFile/starbucks_drinkMenu_expanded.xlsx"

# Load the Excel file
df_starbucks_expanded = spark.read.format("com.crealytics.spark.excel") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("dataAddress", "'starbucks_drinkMenu_expanded.cs'!A1") \
    .load(file_path)

# Display the first few rows
df_starbucks_expanded.show(5)


+--------------------+-------------+-----------------+--------+-------------+-------------+-----------------+-----------+-----------------------+----------------+-----------------+----------+-----------+----------------+----------------+--------------+-----------+-------------+
|   Beverage_category|     Beverage|    Beverage_prep|Calories|Total Fat (g)|Trans Fat (g)|Saturated Fat (g)|Sodium (mg)|Total Carbohydrates (g)|Cholesterol (mg)|Dietary Fibre (g)|Sugars (g)|Protein (g)|Vitamin A (% DV)|Vitamin C (% DV)|Calcium (% DV)|Iron (% DV)|Caffeine (mg)|
+--------------------+-------------+-----------------+--------+-------------+-------------+-----------------+-----------+-----------------------+----------------+-----------------+----------+-----------+----------------+----------------+--------------+-----------+-------------+
|              Coffee|Brewed Coffee|            Short|     3.0|          0.1|          0.0|              0.0|        0.0|                    5.0|             0.0| 

Explore the Dataset
Check the structure of the dataset to understand its schema and columns.


In [31]:
# View the schema
df_starbucks_expanded.printSchema()

# Get summary statistics
df_starbucks_expanded.describe().show()


root
 |-- Beverage_category: string (nullable = true)
 |-- Beverage: string (nullable = true)
 |-- Beverage_prep: string (nullable = true)
 |-- Calories: double (nullable = true)
 |-- Total Fat (g): double (nullable = true)
 |-- Trans Fat (g): double (nullable = true)
 |-- Saturated Fat (g): double (nullable = true)
 |-- Sodium (mg): double (nullable = true)
 |-- Total Carbohydrates (g): double (nullable = true)
 |-- Cholesterol (mg): double (nullable = true)
 |-- Dietary Fibre (g): double (nullable = true)
 |-- Sugars (g): double (nullable = true)
 |-- Protein (g): double (nullable = true)
 |-- Vitamin A (% DV): double (nullable = true)
 |-- Vitamin C (% DV): double (nullable = true)
 |-- Calcium (% DV): double (nullable = true)
 |-- Iron (% DV): double (nullable = true)
 |-- Caffeine (mg): double (nullable = true)

+-------+--------------------+--------------------+-------------+------------------+------------------+------------------+-------------------+-----------------+-----------

Clean the Data
Ensure the data is clean and handle any missing or incorrect values.


**Inspect Null Values** :

**Check the percentage of missing values in each column using .isNull().sum().**

In [32]:
from pyspark.sql.functions import count, when, col

df_starbucks_expanded.select([count(when(col(c).isNull(), c)).alias(c) for c in df_starbucks_expanded.columns]).show()

+-----------------+--------+-------------+--------+-------------+-------------+-----------------+-----------+-----------------------+----------------+-----------------+----------+-----------+----------------+----------------+--------------+-----------+-------------+
|Beverage_category|Beverage|Beverage_prep|Calories|Total Fat (g)|Trans Fat (g)|Saturated Fat (g)|Sodium (mg)|Total Carbohydrates (g)|Cholesterol (mg)|Dietary Fibre (g)|Sugars (g)|Protein (g)|Vitamin A (% DV)|Vitamin C (% DV)|Calcium (% DV)|Iron (% DV)|Caffeine (mg)|
+-----------------+--------+-------------+--------+-------------+-------------+-----------------+-----------+-----------------------+----------------+-----------------+----------+-----------+----------------+----------------+--------------+-----------+-------------+
|                0|       0|            0|       0|            0|            0|                0|          0|                      0|               0|                0|         0|          0|        

**only caffeine column has null value**

In [33]:

from pyspark.sql.functions import col

# Function to get distinct values from all columns
def get_distinct_values_per_column(df):
    distinct_values = {}
    for column in df.columns:
        distinct_values[column] = [row[column] for row in df.select(column).distinct().collect()]
    return distinct_values

# Example usage
distinct_values = get_distinct_values_per_column(df_starbucks_expanded)

# Print distinct values for each column
for column, values in distinct_values.items():
    print(f"Distinct values in column '{column}': {values}")


Distinct values in column 'Beverage_category': ['Tazo® Tea Drinks', 'Coffee', 'Signature Espresso Drinks', 'Classic Espresso Drinks', 'Smoothies', 'Frappuccino® Blended Crème', 'Frappuccino® Light Blended Coffee', 'Frappuccino® Blended Coffee', 'Shaken Iced Beverages']
Distinct values in column 'Beverage': ['Caffè Latte', 'Caramel Macchiato', 'Brewed Coffee', 'Espresso', 'Caffè Mocha (Without Whipped Cream)', 'Caffè Americano', 'Tazo® Green Tea Latte', 'Hot Chocolate (Without Whipped Cream)', 'Tazo® Chai Tea Latte', 'Skinny Latte (Any Flavour)', 'Cappuccino', 'Vanilla Latte (Or Other Flavoured Latte)', 'Caramel Apple Spice (Without Whipped Cream)', 'White Chocolate Mocha (Without Whipped Cream)', 'Tazo® Tea', 'Mocha (Without Whipped Cream)', 'Caramel', 'Iced Brewed Coffee (With Milk & Classic Syrup)', 'Shaken Iced Tazo® Tea Lemonade (With Classic Syrup)', 'Tazo® Full-Leaf Red Tea Latte (Vanilla Rooibos)', 'Mocha', 'Java Chip (Without Whipped Cream)', 'Strawberries & Crème (Without Whip

**Replace "Varies" with Null**

In [34]:
from pyspark.sql.functions import when, col

# Replace "Varies" and "NaN" with null
df_starbucks_cleaned_expanded = df_starbucks_expanded.withColumn(
    "Caffeine (mg)",
    when(col("Caffeine (mg)") == "Varies", None)
    .when(col("Caffeine (mg)") == "NaN", None)
    .otherwise(col("Caffeine (mg)"))
)



Use PySpark's skewness() function to compute the skewness of the column.

To view the skewness of a column in PySpark, you can use the skewness() function from pyspark.sql.functions. Skewness is a measure of the asymmetry of the distribution of values in a dataset.

In [35]:
from pyspark.sql.functions import skewness

df_starbucks_cleaned_expanded.select(skewness("Caffeine (mg)")).show()


+-----------------------+
|skewness(Caffeine (mg))|
+-----------------------+
|     0.8719784462149776|
+-----------------------+



**The skewness value of 0.87 indicates that the Caffeine (mg) column in your dataset has a moderate positive skew. This suggests that most of the data points are concentrated on the lower end of the distribution, but there are some values on the higher end, which extend the right tail.**



Interpretation of Skewness:
* Skewness = 0: The distribution is symmetric.
*   Skewness > 0 (like 0.87): The distribution is positively skewed (right-skewed).
*   Skewness < 0: The distribution is negatively skewed (left-skewed).




**Since the data is moderately skewed, you can consider imputing missing values with the median, as it's less sensitive to skewness than the mean.**

In [36]:
# Check for the count of null values
df_starbucks_cleaned_expanded.filter(df_starbucks_cleaned_expanded["Caffeine (mg)"].isNull()).count()

23

In [37]:
# Check if the column has only one distinct value
df_starbucks_cleaned_expanded.select("Caffeine (mg)").distinct().show()


+-------------+
|Caffeine (mg)|
+-------------+
|         70.0|
|          0.0|
|        410.0|
|        180.0|
|         75.0|
|         NULL|
|        120.0|
|        300.0|
|         25.0|
|        225.0|
|        330.0|
|         85.0|
|         50.0|
|        175.0|
|         10.0|
|         30.0|
|        260.0|
|         95.0|
|         20.0|
|        150.0|
+-------------+
only showing top 20 rows



In [38]:
# Calculate the median value for "Caffeine (mg)"
median_value = df_starbucks_cleaned_expanded.approxQuantile("Caffeine (mg)", [0.5], 0)[0]

In [39]:
median_value

75.0

In [40]:
# Replace missing values with the median value
df_starbucks_cleaned_expanded = df_starbucks_cleaned_expanded.na.fill({"Caffeine (mg)": median_value})

In [41]:

df_starbucks_cleaned_expanded.show()

+--------------------+--------------------+------------------+--------+-------------+-------------+-----------------+-----------+-----------------------+----------------+-----------------+----------+-----------+----------------+----------------+--------------+-----------+-------------+
|   Beverage_category|            Beverage|     Beverage_prep|Calories|Total Fat (g)|Trans Fat (g)|Saturated Fat (g)|Sodium (mg)|Total Carbohydrates (g)|Cholesterol (mg)|Dietary Fibre (g)|Sugars (g)|Protein (g)|Vitamin A (% DV)|Vitamin C (% DV)|Calcium (% DV)|Iron (% DV)|Caffeine (mg)|
+--------------------+--------------------+------------------+--------+-------------+-------------+-----------------+-----------+-----------------------+----------------+-----------------+----------+-----------+----------------+----------------+--------------+-----------+-------------+
|              Coffee|       Brewed Coffee|             Short|     3.0|          0.1|          0.0|              0.0|        0.0|          

In [42]:

# Verify if the missing values were replaced
df_starbucks_cleaned_expanded.filter(df_starbucks_cleaned_expanded["Caffeine (mg)"].isNull()).count()



0

 Verify the Data
Check the column after imputation:

In [43]:
df_starbucks_cleaned_expanded.select("Caffeine (mg)").show()


+-------------+
|Caffeine (mg)|
+-------------+
|        175.0|
|        260.0|
|        330.0|
|        410.0|
|         75.0|
|         75.0|
|         75.0|
|         75.0|
|         75.0|
|         75.0|
|        150.0|
|        150.0|
|        150.0|
|        150.0|
|        150.0|
|        150.0|
|         85.0|
|         85.0|
|         85.0|
|         95.0|
+-------------+
only showing top 20 rows



In [44]:
df_starbucks_cleaned_expanded.printSchema()

root
 |-- Beverage_category: string (nullable = true)
 |-- Beverage: string (nullable = true)
 |-- Beverage_prep: string (nullable = true)
 |-- Calories: double (nullable = true)
 |-- Total Fat (g): double (nullable = true)
 |-- Trans Fat (g): double (nullable = true)
 |-- Saturated Fat (g): double (nullable = true)
 |-- Sodium (mg): double (nullable = true)
 |-- Total Carbohydrates (g): double (nullable = true)
 |-- Cholesterol (mg): double (nullable = true)
 |-- Dietary Fibre (g): double (nullable = true)
 |-- Sugars (g): double (nullable = true)
 |-- Protein (g): double (nullable = true)
 |-- Vitamin A (% DV): double (nullable = true)
 |-- Vitamin C (% DV): double (nullable = true)
 |-- Calcium (% DV): double (nullable = true)
 |-- Iron (% DV): double (nullable = true)
 |-- Caffeine (mg): double (nullable = false)



In [45]:
df_starbucks_cleaned_expanded.describe().show()

+-------+--------------------+--------------------+-------------+------------------+------------------+------------------+-------------------+-----------------+-----------------------+------------------+------------------+------------------+-----------------+-------------------+-------------------+-------------------+-------------------+-----------------+
|summary|   Beverage_category|            Beverage|Beverage_prep|          Calories|     Total Fat (g)|     Trans Fat (g)|  Saturated Fat (g)|      Sodium (mg)|Total Carbohydrates (g)|  Cholesterol (mg)| Dietary Fibre (g)|        Sugars (g)|      Protein (g)|   Vitamin A (% DV)|   Vitamin C (% DV)|     Calcium (% DV)|        Iron (% DV)|    Caffeine (mg)|
+-------+--------------------+--------------------+-------------+------------------+------------------+------------------+-------------------+-----------------+-----------------------+------------------+------------------+------------------+-----------------+-------------------+-----

**Calculate Averages: Compute average metrics for each brand.**

In [46]:
from pyspark.sql.functions import avg

starbucks_summary_exp= df_starbucks_cleaned_expanded.agg(
    avg("Calories").alias("Avg_Calories"),
    avg("Total fat (g)").alias("Avg_T_Fat"),
    avg("Trans Fat (g)").alias("Avg_Ts_Fat"),
    avg("Saturated Fat (g)").alias("Avg_S_Fat"),
    avg("Sodium (mg)").alias("Avg_sodium"),
    avg("Cholesterol (mg)").alias("Avg_Cholesterol"),
    avg("Total Carbohydrates (g)").alias("Avg_Carbohydrates"),
    avg("Dietary Fibre (g)").alias("Avg_Fiber"),
    avg("Sugars (g)").alias("Avg_Sugars"),
    avg("Protein (g)").alias("Avg_Protein"),
    avg("Vitamin A (% DV)").alias("Avg_Vitamin_A"),
    avg("Vitamin C (% DV)").alias("Avg_Vitamin_C"),
    avg("Calcium (% DV)").alias("Avg_Calcium"),
    avg("Iron (% DV)").alias("Avg_Iron"),
    avg("Caffeine (mg)").alias("Avg_Caffeine")
)

starbucks_summary_exp.show()


+------------------+----------------+------------------+-------------------+-----------------+-----------------+------------------+------------------+-----------------+-----------------+-------------------+-------------------+-------------------+-------------------+-----------------+
|      Avg_Calories|       Avg_T_Fat|        Avg_Ts_Fat|          Avg_S_Fat|       Avg_sodium|  Avg_Cholesterol| Avg_Carbohydrates|         Avg_Fiber|       Avg_Sugars|      Avg_Protein|      Avg_Vitamin_A|      Avg_Vitamin_C|        Avg_Calcium|           Avg_Iron|     Avg_Caffeine|
+------------------+----------------+------------------+-------------------+-----------------+-----------------+------------------+------------------+-----------------+-----------------+-------------------+-------------------+-------------------+-------------------+-----------------+
|193.87190082644628|190.300826446281|1.3070247933884296|0.03760330578512397|6.363636363636363|35.99173553719008|128.88429752066116|0.805785123966

**Mcdonalds dataset**

In [47]:
file_path = "/content/drive/MyDrive/Internship_projectFile/menu.xlsx"

# Load the Excel file
df_Mcdonalds = spark.read.format("com.crealytics.spark.excel") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("dataAddress", "menu.csv!A1") \
    .load(file_path)

# Display the first few rows
df_Mcdonalds.show(5)

+---------+--------------------+--------------+--------+-----------------+---------+-------------------------+-------------+-----------------------------+---------+-----------+---------------------------+------+----------------------+-------------+-----------------------------+-------------+-----------------------------+------+-------+-------------------------+-------------------------+-----------------------+--------------------+
| Category|                Item|  Serving Size|Calories|Calories from Fat|Total Fat|Total Fat (% Daily Value)|Saturated Fat|Saturated Fat (% Daily Value)|Trans Fat|Cholesterol|Cholesterol (% Daily Value)|Sodium|Sodium (% Daily Value)|Carbohydrates|Carbohydrates (% Daily Value)|Dietary Fiber|Dietary Fiber (% Daily Value)|Sugars|Protein|Vitamin A (% Daily Value)|Vitamin C (% Daily Value)|Calcium (% Daily Value)|Iron (% Daily Value)|
+---------+--------------------+--------------+--------+-----------------+---------+-------------------------+-------------+------

In [48]:
df_Mcdonalds.select("Category").distinct().show()

+------------------+
|          Category|
+------------------+
|    Chicken & Fish|
|       Beef & Pork|
|            Salads|
|         Breakfast|
|         Beverages|
|    Snacks & Sides|
|          Desserts|
|Smoothies & Shakes|
|      Coffee & Tea|
+------------------+



In [49]:
# Define the categories for food and drinks
food_categories = ['Breakfast', 'Beef & Pork', 'Chicken & Fish', 'Salads', 'Snacks & Sides', 'Desserts']
drink_categories = ['Beverages', 'Coffee & Tea', 'Smoothies & Shakes']

# Separate food data based on the food categories
food_df_McD = df_Mcdonalds.filter(df_Mcdonalds['Category'].isin(food_categories))

# Separate drink data based on the drink categories
drink_df_McD = df_Mcdonalds.filter(df_Mcdonalds['Category'].isin(drink_categories))

# Show the first few rows of the food data
print("Food Data:")
food_df_McD.show()

# Show the first few rows of the drink data
print("Drink Data:")
drink_df_McD.show()


Food Data:
+---------+--------------------+--------------+--------+-----------------+---------+-------------------------+-------------+-----------------------------+---------+-----------+---------------------------+------+----------------------+-------------+-----------------------------+-------------+-----------------------------+------+-------+-------------------------+-------------------------+-----------------------+--------------------+
| Category|                Item|  Serving Size|Calories|Calories from Fat|Total Fat|Total Fat (% Daily Value)|Saturated Fat|Saturated Fat (% Daily Value)|Trans Fat|Cholesterol|Cholesterol (% Daily Value)|Sodium|Sodium (% Daily Value)|Carbohydrates|Carbohydrates (% Daily Value)|Dietary Fiber|Dietary Fiber (% Daily Value)|Sugars|Protein|Vitamin A (% Daily Value)|Vitamin C (% Daily Value)|Calcium (% Daily Value)|Iron (% Daily Value)|
+---------+--------------------+--------------+--------+-----------------+---------+-------------------------+---------

In [50]:
# View the schema
drink_df_McD.printSchema()
food_df_McD.printSchema()


root
 |-- Category: string (nullable = true)
 |-- Item: string (nullable = true)
 |-- Serving Size: string (nullable = true)
 |-- Calories: double (nullable = true)
 |-- Calories from Fat: double (nullable = true)
 |-- Total Fat: double (nullable = true)
 |-- Total Fat (% Daily Value): double (nullable = true)
 |-- Saturated Fat: double (nullable = true)
 |-- Saturated Fat (% Daily Value): double (nullable = true)
 |-- Trans Fat: double (nullable = true)
 |-- Cholesterol: double (nullable = true)
 |-- Cholesterol (% Daily Value): double (nullable = true)
 |-- Sodium: double (nullable = true)
 |-- Sodium (% Daily Value): double (nullable = true)
 |-- Carbohydrates: double (nullable = true)
 |-- Carbohydrates (% Daily Value): double (nullable = true)
 |-- Dietary Fiber: double (nullable = true)
 |-- Dietary Fiber (% Daily Value): double (nullable = true)
 |-- Sugars: double (nullable = true)
 |-- Protein: double (nullable = true)
 |-- Vitamin A (% Daily Value): double (nullable = true)
 

In [51]:
# checking null values in columns
from pyspark.sql.functions import count, when, col

drink_df_McD.select([count(when(col(c).isNull(), c)).alias(c) for c in drink_df_McD.columns]).show()
food_df_McD.select([count(when(col(c).isNull(), c)).alias(c) for c in food_df_McD.columns]).show()

+--------+----+------------+--------+-----------------+---------+-------------------------+-------------+-----------------------------+---------+-----------+---------------------------+------+----------------------+-------------+-----------------------------+-------------+-----------------------------+------+-------+-------------------------+-------------------------+-----------------------+--------------------+
|Category|Item|Serving Size|Calories|Calories from Fat|Total Fat|Total Fat (% Daily Value)|Saturated Fat|Saturated Fat (% Daily Value)|Trans Fat|Cholesterol|Cholesterol (% Daily Value)|Sodium|Sodium (% Daily Value)|Carbohydrates|Carbohydrates (% Daily Value)|Dietary Fiber|Dietary Fiber (% Daily Value)|Sugars|Protein|Vitamin A (% Daily Value)|Vitamin C (% Daily Value)|Calcium (% Daily Value)|Iron (% Daily Value)|
+--------+----+------------+--------+-----------------+---------+-------------------------+-------------+-----------------------------+---------+-----------+-----------

In [52]:
drink_df_McD.describe().show()
food_df_McD.describe().show()

+-------+------------------+--------------------+-----------------+------------------+-----------------+-----------------+-------------------------+-----------------+-----------------------------+-------------------+------------------+---------------------------+------------------+----------------------+------------------+-----------------------------+------------------+-----------------------------+------------------+-----------------+-------------------------+-------------------------+-----------------------+--------------------+
|summary|          Category|                Item|     Serving Size|          Calories|Calories from Fat|        Total Fat|Total Fat (% Daily Value)|    Saturated Fat|Saturated Fat (% Daily Value)|          Trans Fat|       Cholesterol|Cholesterol (% Daily Value)|            Sodium|Sodium (% Daily Value)|     Carbohydrates|Carbohydrates (% Daily Value)|     Dietary Fiber|Dietary Fiber (% Daily Value)|            Sugars|          Protein|Vitamin A (% Daily Val

In [53]:
# calculating averages
from pyspark.sql.functions import avg

drink_summary_McD = drink_df_McD.agg(
  avg("Calories").alias("Avg_Calories"),
  avg("Calories from Fat").alias("Avg_Calories_from_Fat"),
  avg("Total fat").alias("Avg_T_Fat"),
  avg("Total fat (% Daily Value)").alias("Avg_T_Fat_DV"),
  avg("Saturated Fat").alias("Avg_S_Fat"),
  avg("Saturated Fat (% Daily Value)").alias("Avg_S_Fat_DV"),
  avg("Trans Fat").alias("Avg_Ts_Fat"),
  avg("Sodium").alias("Avg_sodium"),
  avg("Sodium (% Daily Value)").alias("Avg_sodium_DV"),
  avg("Cholesterol").alias("Avg_Cholesterol"),
  avg("Cholesterol (% Daily Value)").alias("Avg_Cholesterol_DV"),
  avg("Carbohydrates").alias("Avg_Carbohydrates"),
  avg("Carbohydrates (% Daily Value)").alias("Avg_Carbohydrates_DV"),
  avg("Dietary Fiber").alias("Avg_Fiber"),
  avg("Dietary Fiber (% Daily Value)").alias("Avg_Fiber_DV"),
  avg("Sugars").alias("Avg_Sugars"),
  avg("Protein").alias("Avg_Protein"),
  avg("Vitamin A (% Daily Value)").alias("Avg_Vitamin_A"),
  avg("Vitamin C (% Daily Value)").alias("Avg_Vitamin_C"),
  avg("Calcium (% Daily Value)").alias("Avg_Calcium"),
  avg("Iron (% Daily Value)").alias("Avg_Iron"))



food_summary_McD = food_df_McD.agg(
  avg("Calories").alias("Avg_Calories"),
  avg("Calories from Fat").alias("Avg_Calories_from_Fat"),
  avg("Total fat").alias("Avg_T_Fat"),
  avg("Total fat (% Daily Value)").alias("Avg_T_Fat_DV"),
  avg("Saturated Fat").alias("Avg_S_Fat"),
  avg("Saturated Fat (% Daily Value)").alias("Avg_S_Fat_DV"),
  avg("Trans Fat").alias("Avg_Ts_Fat"),
  avg("Sodium").alias("Avg_sodium"),
  avg("Sodium (% Daily Value)").alias("Avg_sodium_DV"),
  avg("Cholesterol").alias("Avg_Cholesterol"),
  avg("Cholesterol (% Daily Value)").alias("Avg_Cholesterol_DV"),
  avg("Carbohydrates").alias("Avg_Carbohydrates"),
  avg("Carbohydrates (% Daily Value)").alias("Avg_Carbohydrates_DV"),
  avg("Dietary Fiber").alias("Avg_Fiber"),
  avg("Dietary Fiber (% Daily Value)").alias("Avg_Fiber_DV"),
  avg("Sugars").alias("Avg_Sugars"),
  avg("Protein").alias("Avg_Protein"),
  avg("Vitamin A (% Daily Value)").alias("Avg_Vitamin_A"),
  avg("Vitamin C (% Daily Value)").alias("Avg_Vitamin_C"),
  avg("Calcium (% Daily Value)").alias("Avg_Calcium"),
  avg("Iron (% Daily Value)").alias("Avg_Iron"))



drink_summary_McD.show()
food_summary_McD.show()

+------------------+---------------------+-----------------+------------------+---------+------------------+----------+------------------+------------------+------------------+------------------+------------------+--------------------+------------------+------------+----------+-----------+------------------+-----------------+------------------+------------------+
|      Avg_Calories|Avg_Calories_from_Fat|        Avg_T_Fat|      Avg_T_Fat_DV|Avg_S_Fat|      Avg_S_Fat_DV|Avg_Ts_Fat|        Avg_sodium|     Avg_sodium_DV|   Avg_Cholesterol|Avg_Cholesterol_DV| Avg_Carbohydrates|Avg_Carbohydrates_DV|         Avg_Fiber|Avg_Fiber_DV|Avg_Sugars|Avg_Protein|     Avg_Vitamin_A|    Avg_Vitamin_C|       Avg_Calcium|          Avg_Iron|
+------------------+---------------------+-----------------+------------------+---------+------------------+----------+------------------+------------------+------------------+------------------+------------------+--------------------+------------------+------------+-

**Starbucks nutrition drinks dataset**

In [54]:
file_path = "/content/drive/MyDrive/Internship_projectFile/starbucks-menu-nutrition-drinks.xlsx"

# Load the Excel file
df_starbucks_drinks = spark.read.format("com.crealytics.spark.excel") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("dataAddress", "starbucks-menu-nutrition-drinks!A1") \
    .load(file_path)

# Display the first few rows
df_starbucks_drinks.show(5)

+--------+-------+---------+---------+-------+------+
|Calories|Fat (g)|Carb. (g)|Fiber (g)|Protein|Sodium|
+--------+-------+---------+---------+-------+------+
|      45|      0|       11|        0|      0|    10|
|       -|      -|        -|        -|      -|     -|
|       -|      -|        -|        -|      -|     -|
|      80|      0|       18|        1|      0|    10|
|      60|      0|       14|        1|      0|    10|
+--------+-------+---------+---------+-------+------+
only showing top 5 rows



In [55]:
# View the schema
df_starbucks_drinks.printSchema()

# Rename the column 'Carb. (g)' to 'Carb_g'
df_starbucks_drinks = df_starbucks_drinks.withColumnRenamed("Carb. (g)", "Carb_g")

# Now try the describe function again
df_starbucks_drinks.describe().show()

root
 |-- Calories: string (nullable = true)
 |-- Fat (g): string (nullable = true)
 |-- Carb. (g): string (nullable = true)
 |-- Fiber (g): string (nullable = true)
 |-- Protein: string (nullable = true)
 |-- Sodium: string (nullable = true)

+-------+------------------+------------------+------------------+-------------------+------------------+-----------------+
|summary|          Calories|           Fat (g)|            Carb_g|          Fiber (g)|           Protein|           Sodium|
+-------+------------------+------------------+------------------+-------------------+------------------+-----------------+
|  count|               177|               177|               177|                177|               177|              177|
|   mean|135.16304347826087|2.3369565217391304| 24.73913043478261|0.44565217391304346|4.1521739130434785|57.93478260869565|
| stddev| 93.85355405213949| 3.887927705634804|15.214849286009594| 1.4170401782230873| 5.691587743885265| 68.3846272082678|
|    min|   

In [56]:
# checking for null values
from pyspark.sql.functions import count, when, col

df_starbucks_drinks.select([count(when(col(c).isNull(), c)).alias(c) for c in df_starbucks_drinks.columns]).show()

+--------+-------+------+---------+-------+------+
|Calories|Fat (g)|Carb_g|Fiber (g)|Protein|Sodium|
+--------+-------+------+---------+-------+------+
|       0|      0|     0|        0|      0|     0|
+--------+-------+------+---------+-------+------+



In [57]:
from pyspark.sql.functions import col, when, count

# Check if '-' exists in any column
df_starbucks_drinks.select([
    count(when(col(c).contains('-'), c)).alias(c) for c in df_starbucks_drinks.columns
]).show()


+--------+-------+------+---------+-------+------+
|Calories|Fat (g)|Carb_g|Fiber (g)|Protein|Sodium|
+--------+-------+------+---------+-------+------+
|      85|     85|    85|       85|     85|    85|
+--------+-------+------+---------+-------+------+



**In dataset, missing values are represented by hyphens ("-"). We will replace these hyphens with null values for proper handling.**

In [58]:
from pyspark.sql.functions import when, col

# Replace hyphens ("-") with null
df_starbucks_drinks_cleaned = df_starbucks_drinks.select([when(col(c) == "-", None).otherwise(col(c)).alias(c) for c in df_starbucks_drinks.columns])

# Show the cleaned data
df_starbucks_drinks_cleaned.show(5)


+--------+-------+------+---------+-------+------+
|Calories|Fat (g)|Carb_g|Fiber (g)|Protein|Sodium|
+--------+-------+------+---------+-------+------+
|      45|      0|    11|        0|      0|    10|
|    NULL|   NULL|  NULL|     NULL|   NULL|  NULL|
|    NULL|   NULL|  NULL|     NULL|   NULL|  NULL|
|      80|      0|    18|        1|      0|    10|
|      60|      0|    14|        1|      0|    10|
+--------+-------+------+---------+-------+------+
only showing top 5 rows



In [59]:
df_starbucks_drinks_cleaned.select([count(when(col(c).isNull(), c)).alias(c) for c in df_starbucks_drinks_cleaned.columns]).show()

+--------+-------+------+---------+-------+------+
|Calories|Fat (g)|Carb_g|Fiber (g)|Protein|Sodium|
+--------+-------+------+---------+-------+------+
|      85|     85|    85|       85|     85|    85|
+--------+-------+------+---------+-------+------+



In [60]:
df_starbucks_drinks_cleaned.printSchema()

root
 |-- Calories: string (nullable = true)
 |-- Fat (g): string (nullable = true)
 |-- Carb_g: string (nullable = true)
 |-- Fiber (g): string (nullable = true)
 |-- Protein: string (nullable = true)
 |-- Sodium: string (nullable = true)



**Convert Data Types
Convert numerical columns (Calories, Fat (g), etc.) from strings to numeric types (float or integer).**

In [61]:
numeric_columns = ["Calories", "Fat (g)", "Carb_g", "Fiber (g)", "Protein", "Sodium"]

# Convert columns to numeric types
for col_name in numeric_columns:
    df_starbucks_drinks_cleaned = df_starbucks_drinks_cleaned.withColumn(col_name, col(col_name).cast("float"))

df_starbucks_drinks_cleaned.printSchema()


root
 |-- Calories: float (nullable = true)
 |-- Fat (g): float (nullable = true)
 |-- Carb_g: float (nullable = true)
 |-- Fiber (g): float (nullable = true)
 |-- Protein: float (nullable = true)
 |-- Sodium: float (nullable = true)



**Calculate Skewness for Multiple Columns
If you want to compute skewness for multiple numerical columns, you can use a loop:**

In [62]:

# List of columns to calculate skewness
numeric_columns = ["Calories", "Fat (g)", "Carb_g", "Fiber (g)", "Protein", "Sodium"]

# Calculate skewness for each column
for col_name in numeric_columns:
    skew = df_starbucks_drinks_cleaned.select(skewness(col_name)).collect()[0][0]
    print(f"Skewness of {col_name}: {skew}")
#The f before the string indicates that it’s a formatted string

Skewness of Calories: 0.7140768008747559
Skewness of Fat (g): 2.900588948455683
Skewness of Carb_g: 0.16752090523230811
Skewness of Fiber (g): 3.6756343080776785
Skewness of Protein: 1.166078243738295
Skewness of Sodium: 0.9372011266675181


* Calories (0.71): Moderately right-skewed.
* Fat (g) (2.9): Highly right-skewed.
* Carb_g (0.16): Close to symmetric.
* Fiber (g) (3.68): Highly right-skewed.
* Protein (1.17): Moderately right-skewed.
* Sodium (0.94): Moderately right-skewed.




A high skewness (above 1 or below -1) suggests a significant asymmetry in the data, indicating a need for transformations.



**Analyze the Skewness**
* Low Skewness (close to 0): For Calories, Carb_g, and Sodium, the skewness is low, indicating a relatively symmetric distribution. For such columns, you can fill missing values with the mean or median.

* High Positive Skewness (>1): For Fat (g), Fiber (g), and Protein, the skewness is high, indicating right-skewed distributions. For such columns, the median is a better choice because it is less affected by outliers.

In [63]:
df_starbucks_drinks_cleaned.show()

+--------+-------+------+---------+-------+------+
|Calories|Fat (g)|Carb_g|Fiber (g)|Protein|Sodium|
+--------+-------+------+---------+-------+------+
|    45.0|    0.0|  11.0|      0.0|    0.0|  10.0|
|    NULL|   NULL|  NULL|     NULL|   NULL|  NULL|
|    NULL|   NULL|  NULL|     NULL|   NULL|  NULL|
|    80.0|    0.0|  18.0|      1.0|    0.0|  10.0|
|    60.0|    0.0|  14.0|      1.0|    0.0|  10.0|
|    NULL|   NULL|  NULL|     NULL|   NULL|  NULL|
|    NULL|   NULL|  NULL|     NULL|   NULL|  NULL|
|    NULL|   NULL|  NULL|     NULL|   NULL|  NULL|
|   110.0|    0.0|  28.0|      0.0|    0.0|   5.0|
|     0.0|    0.0|   0.0|      0.0|    0.0|   0.0|
|    NULL|   NULL|  NULL|     NULL|   NULL|  NULL|
|   130.0|    2.5|  21.0|      0.0|    5.0|  65.0|
|   140.0|    2.5|  23.0|      0.0|    5.0|  90.0|
|   130.0|    2.5|  21.0|      0.0|    5.0|  65.0|
|    NULL|   NULL|  NULL|     NULL|   NULL|  NULL|
|    NULL|   NULL|  NULL|     NULL|   NULL|  NULL|
|    NULL|   NULL|  NULL|     N

In [64]:
from pyspark.sql.functions import mean, expr, median

# Fill low-skewness columns with mean

LS_starbucks_drinks = ["Calories", "Carb_g", "Sodium"]
for col_name in LS_starbucks_drinks:
    mean_value = df_starbucks_drinks_cleaned.select(mean(col_name)).collect()[0][0]
    df_starbucks_drinks_cleaned= df_starbucks_drinks_cleaned.fillna({col_name: mean_value})

# Fill high-skewness columns with median
HS_starbucks_drinks = ["Fat (g)", "Fiber (g)", "Protein"]
for col_name in HS_starbucks_drinks:
    median_value = df_starbucks_drinks_cleaned.approxQuantile(col_name, [0.5], 0)[0]
    df_starbucks_drinks_cleaned = df_starbucks_drinks_cleaned.fillna({col_name: median_value})

In [65]:
df_starbucks_drinks_cleaned.show()


+---------+-------+--------+---------+-------+---------+
| Calories|Fat (g)|  Carb_g|Fiber (g)|Protein|   Sodium|
+---------+-------+--------+---------+-------+---------+
|     45.0|    0.0|    11.0|      0.0|    0.0|     10.0|
|135.16304|    0.0|24.73913|      0.0|    1.0|57.934784|
|135.16304|    0.0|24.73913|      0.0|    1.0|57.934784|
|     80.0|    0.0|    18.0|      1.0|    0.0|     10.0|
|     60.0|    0.0|    14.0|      1.0|    0.0|     10.0|
|135.16304|    0.0|24.73913|      0.0|    1.0|57.934784|
|135.16304|    0.0|24.73913|      0.0|    1.0|57.934784|
|135.16304|    0.0|24.73913|      0.0|    1.0|57.934784|
|    110.0|    0.0|    28.0|      0.0|    0.0|      5.0|
|      0.0|    0.0|     0.0|      0.0|    0.0|      0.0|
|135.16304|    0.0|24.73913|      0.0|    1.0|57.934784|
|    130.0|    2.5|    21.0|      0.0|    5.0|     65.0|
|    140.0|    2.5|    23.0|      0.0|    5.0|     90.0|
|    130.0|    2.5|    21.0|      0.0|    5.0|     65.0|
|135.16304|    0.0|24.73913|   

In [66]:
df_starbucks_drinks_cleaned.printSchema()

root
 |-- Calories: float (nullable = false)
 |-- Fat (g): float (nullable = false)
 |-- Carb_g: float (nullable = false)
 |-- Fiber (g): float (nullable = false)
 |-- Protein: float (nullable = false)
 |-- Sodium: float (nullable = false)



In [67]:

df_starbucks_drinks_cleaned.show()

+---------+-------+--------+---------+-------+---------+
| Calories|Fat (g)|  Carb_g|Fiber (g)|Protein|   Sodium|
+---------+-------+--------+---------+-------+---------+
|     45.0|    0.0|    11.0|      0.0|    0.0|     10.0|
|135.16304|    0.0|24.73913|      0.0|    1.0|57.934784|
|135.16304|    0.0|24.73913|      0.0|    1.0|57.934784|
|     80.0|    0.0|    18.0|      1.0|    0.0|     10.0|
|     60.0|    0.0|    14.0|      1.0|    0.0|     10.0|
|135.16304|    0.0|24.73913|      0.0|    1.0|57.934784|
|135.16304|    0.0|24.73913|      0.0|    1.0|57.934784|
|135.16304|    0.0|24.73913|      0.0|    1.0|57.934784|
|    110.0|    0.0|    28.0|      0.0|    0.0|      5.0|
|      0.0|    0.0|     0.0|      0.0|    0.0|      0.0|
|135.16304|    0.0|24.73913|      0.0|    1.0|57.934784|
|    130.0|    2.5|    21.0|      0.0|    5.0|     65.0|
|    140.0|    2.5|    23.0|      0.0|    5.0|     90.0|
|    130.0|    2.5|    21.0|      0.0|    5.0|     65.0|
|135.16304|    0.0|24.73913|   

In [68]:
# Verify if the missing values were replaced

df_starbucks_drinks_cleaned.select([count(when(col(c).isNull(), c)).alias(c) for c in df_starbucks_drinks_cleaned.columns]).show()

+--------+-------+------+---------+-------+------+
|Calories|Fat (g)|Carb_g|Fiber (g)|Protein|Sodium|
+--------+-------+------+---------+-------+------+
|       0|      0|     0|        0|      0|     0|
+--------+-------+------+---------+-------+------+



In [69]:
df_starbucks_drinks_cleaned.describe().show()

+-------+------------------+------------------+-----------------+-------------------+-----------------+-----------------+
|summary|          Calories|           Fat (g)|           Carb_g|          Fiber (g)|          Protein|           Sodium|
+-------+------------------+------------------+-----------------+-------------------+-----------------+-----------------+
|  count|               177|               177|              177|                177|              177|              177|
|   mean|135.16304188528977|1.2146892655367232|24.73913023566122|0.23163841807909605|2.638418079096045|57.93478324588409|
| stddev| 67.48621721018243|  3.03094064298603| 10.9403701767693| 1.0431127609956792| 4.38674317291608|49.17256306618584|
|    min|               0.0|               0.0|              0.0|                0.0|              0.0|              0.0|
|    max|             430.0|              26.0|             64.0|                8.0|             20.0|            240.0|
+-------+---------------

**Starbucks food data**

In [70]:
file_path = "/content/drive/MyDrive/Internship_projectFile/starbucks-menu-nutrition-food.xlsx"

# Load the Excel file
df_starbucks_food = spark.read.format("com.crealytics.spark.excel") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("dataAddress", "starbucks-menu-nutrition-food.c!A1") \
    .load(file_path)

# Display the first few rows
df_starbucks_food.show(5)

+--------+-------+---------+---------+-----------+
|Calories|Fat (g)|Carb. (g)|Fiber (g)|Protein (g)|
+--------+-------+---------+---------+-----------+
|   300.0|    5.0|     50.0|      3.0|       12.0|
|   380.0|    6.0|     70.0|      7.0|       10.0|
|   410.0|   22.0|     45.0|      3.0|       10.0|
|   460.0|   23.0|     56.0|      2.0|        7.0|
|   420.0|   22.0|     52.0|      2.0|        6.0|
+--------+-------+---------+---------+-----------+
only showing top 5 rows



In [71]:
# Rename the column 'Carb. (g)' to 'Carb_g'
df_starbucks_food = df_starbucks_food.withColumnRenamed("Carb. (g)", "Carb_g")

In [72]:
# View the schema
df_starbucks_food.printSchema()

# Get summary statistics
df_starbucks_food.describe().show()

root
 |-- Calories: double (nullable = true)
 |-- Fat (g): double (nullable = true)
 |-- Carb_g: double (nullable = true)
 |-- Fiber (g): double (nullable = true)
 |-- Protein (g): double (nullable = true)

+-------+-----------------+------------------+-----------------+-----------------+------------------+
|summary|         Calories|           Fat (g)|           Carb_g|        Fiber (g)|       Protein (g)|
+-------+-----------------+------------------+-----------------+-----------------+------------------+
|  count|              113|               113|              113|              113|               113|
|   mean|356.6371681415929|16.353982300884955|41.48672566371681|2.849557522123894|11.469026548672566|
| stddev|127.7106846091491| 8.297396702303573|15.79676449932966| 2.88846609978516|  8.46323012935978|
|    min|             90.0|               0.0|              5.0|              0.0|               1.0|
|    max|            650.0|              37.0|             80.0|             21

In [73]:
from pyspark.sql.functions import avg

df_starbucks_food_summary = df_starbucks_food.agg(
    avg("Calories").alias("Avg_Calories"),
    avg("fat (g)").alias("Avg_Fat"),
    avg("Carb_g").alias("Avg_Carb_g"),
    avg("Fiber (g)").alias("Avg_Fiber"),
    avg("Protein (g)").alias("Avg_Protein"),
)

In [74]:
df_starbucks_food_summary.show()

+-----------------+------------------+-----------------+-----------------+------------------+
|     Avg_Calories|           Avg_Fat|       Avg_Carb_g|        Avg_Fiber|       Avg_Protein|
+-----------------+------------------+-----------------+-----------------+------------------+
|356.6371681415929|16.353982300884955|41.48672566371681|2.849557522123894|11.469026548672566|
+-----------------+------------------+-----------------+-----------------+------------------+



# **Analysis of starbucks food and macdonalds food**

In [75]:
starbucks_summary_food = df_starbucks_food.agg(
    avg("calories").alias("Avg_Calories"),
    avg("carb_g").alias("Avg_Carbohydrates"),
    avg("protein (g)").alias("Avg_Protein"),
    avg("Fat (g)").alias("Avg_Fat"),
    avg("Fiber (g)").alias("Avg_Fibre"))

macdonalds_summary_food = food_summary_McD.agg(
    avg("Avg_Calories").alias("Avg_Calories"),
    avg("Avg_Carbohydrates").alias("Avg_Carbohydrates"),
    avg("Avg_Protein").alias("Avg_Protein"),
    avg("Avg_T_Fat").alias("Avg_Fat"),
    avg("Avg_Fiber").alias("Avg_Fibre"))

In [76]:
# Show summaries
starbucks_summary_food.show()
macdonalds_summary_food.show()

+-----------------+-----------------+------------------+------------------+-----------------+
|     Avg_Calories|Avg_Carbohydrates|       Avg_Protein|           Avg_Fat|        Avg_Fibre|
+-----------------+-----------------+------------------+------------------+-----------------+
|356.6371681415929|41.48672566371681|11.469026548672566|16.353982300884955|2.849557522123894|
+-----------------+-----------------+------------------+------------------+-----------------+

+------------------+-----------------+-----------------+------------------+---------+
|      Avg_Calories|Avg_Carbohydrates|      Avg_Protein|           Avg_Fat|Avg_Fibre|
+------------------+-----------------+-----------------+------------------+---------+
|462.09090909090907|43.36363636363637|20.78181818181818|22.936363636363637|      2.8|
+------------------+-----------------+-----------------+------------------+---------+



**Combine the Summaries for Comparison
After calculating the averages for both datasets, merge the summaries to enable side-by-side comparisons.**

In [77]:
from pyspark.sql.functions import avg

# Calculate the average values for the common columns in df_starbucks_food_summary
avg_values_starbucks = df_starbucks_food_summary.select(
    avg('Avg_Calories').alias('Avg_Calories'),
    avg('Avg_Fat').alias('Avg_Fat'),
    avg('Avg_Carb_g').alias('Avg_Carb_g'),
    avg('Avg_Fiber').alias('Avg_Fiber'),
    avg('Avg_Protein').alias('Avg_Protein')
).collect()

# Calculate the average values for the common columns in food_summary_McD

avg_values_mcd = food_summary_McD.select(
    avg('Avg_Calories').alias('Avg_Calories'),
    avg('Avg_T_Fat').alias('Avg_Fat'),
    avg('Avg_Carbohydrates').alias('Avg_Carb_g'),
    avg('Avg_Fiber').alias('Avg_Fiber'),
    avg('Avg_Protein').alias('Avg_Protein')
).collect()

# Extract values from both DataFrames
avg_values_starbucks = avg_values_starbucks[0]
avg_values_mcd = avg_values_mcd[0]

# Prepare the comparison table
# Use attribute access or indexing to get values from Row objects
comparison_data = [
    ("Avg_Calories", avg_values_starbucks[0], avg_values_mcd[0]),  # Accessing using index
    ("Avg_Fat", avg_values_starbucks[1], avg_values_mcd[1]),  # Accessing using index
    ("Avg_Carb_g", avg_values_starbucks[2], avg_values_mcd[2]),  # Accessing using index
    ("Avg_Fiber", avg_values_starbucks[3], avg_values_mcd[3]),  # Accessing using index
    ("Avg_Protein", avg_values_starbucks[4], avg_values_mcd[4]),  # Accessing using index
]

# Convert the comparison data into a DataFrame
comparison_df = spark.createDataFrame(comparison_data, ["Nutritional Factor", "Starbucks Average", "McD Average"])

# Show the comparison table
comparison_df.show()

+------------------+------------------+------------------+
|Nutritional Factor| Starbucks Average|       McD Average|
+------------------+------------------+------------------+
|      Avg_Calories| 356.6371681415929|462.09090909090907|
|           Avg_Fat|16.353982300884955|22.936363636363637|
|        Avg_Carb_g| 41.48672566371681| 43.36363636363637|
|         Avg_Fiber| 2.849557522123894|               2.8|
|       Avg_Protein|11.469026548672566| 20.78181818181818|
+------------------+------------------+------------------+



1. Calories
* Starbucks Average: 356.64 kcal
* McDonald's Average: 462.09 kcal
* Healthier Option: Starbucks
Insight: Starbucks meals are significantly lower in calories, making them a better choice for calorie-conscious individuals or those aiming to manage their weight.

2. Fat
* Starbucks Average: 16.35 g
* McDonald's Average: 22.94 g
* Healthier Option: Starbucks
Insight: Starbucks food contains less fat than McDonald's, which might appeal to people looking for lower-fat meal options to reduce risks associated with excessive fat intake.

3. Carbohydrates
* Starbucks Average: 41.49 g
* McDonald's Average: 43.36 g
* Healthier Option: Starbucks
Insight: Both brands offer meals with similar carbohydrate content, but Starbucks is slightly lower. This might make it a marginally better option for those managing carbohydrate intake, such as diabetics.

4. Fiber
* Starbucks Average: 2.85 g
* McDonald's Average: 2.8 g
* Healthier Option: Starbucks
Insight: Both brands are comparable in fiber content, with Starbucks providing a minimal edge. Higher fiber is generally beneficial for digestion and overall health.


5. Protein
* Starbucks Average: 11.47 g
* McDonald's Average: 20.78 g
* Healthier Option: McDonald's
Insight: McDonald's meals have almost double the protein content compared to Starbucks. This makes McDonald's a better option for individuals looking to increase their protein intake, such as athletes or those on high-protein diets.


* Starbucks offers meals that are generally lower in calories, fat, and carbohydrates, making it a healthier choice for individuals prioritizing these factors.
* McDonald's stands out in protein content, which can be beneficial for muscle building and satiety but comes at the cost of higher calories and fat.
* Fiber content is nearly identical for both, so neither brand offers a clear advantage in this area.


**Overall Insights:**
* For Weight Loss or Maintenance: Starbucks is the better choice due to lower calorie and fat content.
* For Active or Protein-Focused Diets: McDonald's may be preferable because of its higher protein content.
* For Balanced Choices: Evaluate specific items on both menus as the averages suggest trade-offs between calories, fat, and protein.**

# **Drinks comaprision**


1.**comparision between starbucks drink expanded data and mcdonalds data**

In [78]:
starbucks_summary_exp.printSchema()

root
 |-- Avg_Calories: double (nullable = true)
 |-- Avg_T_Fat: double (nullable = true)
 |-- Avg_Ts_Fat: double (nullable = true)
 |-- Avg_S_Fat: double (nullable = true)
 |-- Avg_sodium: double (nullable = true)
 |-- Avg_Cholesterol: double (nullable = true)
 |-- Avg_Carbohydrates: double (nullable = true)
 |-- Avg_Fiber: double (nullable = true)
 |-- Avg_Sugars: double (nullable = true)
 |-- Avg_Protein: double (nullable = true)
 |-- Avg_Vitamin_A: double (nullable = true)
 |-- Avg_Vitamin_C: double (nullable = true)
 |-- Avg_Calcium: double (nullable = true)
 |-- Avg_Iron: double (nullable = true)
 |-- Avg_Caffeine: double (nullable = true)



In [79]:
drink_summary_McD.printSchema()

root
 |-- Avg_Calories: double (nullable = true)
 |-- Avg_Calories_from_Fat: double (nullable = true)
 |-- Avg_T_Fat: double (nullable = true)
 |-- Avg_T_Fat_DV: double (nullable = true)
 |-- Avg_S_Fat: double (nullable = true)
 |-- Avg_S_Fat_DV: double (nullable = true)
 |-- Avg_Ts_Fat: double (nullable = true)
 |-- Avg_sodium: double (nullable = true)
 |-- Avg_sodium_DV: double (nullable = true)
 |-- Avg_Cholesterol: double (nullable = true)
 |-- Avg_Cholesterol_DV: double (nullable = true)
 |-- Avg_Carbohydrates: double (nullable = true)
 |-- Avg_Carbohydrates_DV: double (nullable = true)
 |-- Avg_Fiber: double (nullable = true)
 |-- Avg_Fiber_DV: double (nullable = true)
 |-- Avg_Sugars: double (nullable = true)
 |-- Avg_Protein: double (nullable = true)
 |-- Avg_Vitamin_A: double (nullable = true)
 |-- Avg_Vitamin_C: double (nullable = true)
 |-- Avg_Calcium: double (nullable = true)
 |-- Avg_Iron: double (nullable = true)



In [80]:
from pyspark.sql import functions as F

# Calculate the average values for the common columns in df_starbucks_summary - expand & drink_summary_McD
avg_values_starbucks = starbucks_summary_exp.select(
    F.avg('Avg_Calories').alias('Avg_Calories'),
    F.avg('Avg_T_Fat').alias('Avg_T_Fat'),
    F.avg('Avg_S_Fat').alias('Avg_S_Fat'),
    F.avg('Avg_Ts_Fat').alias('Avg_Ts_Fat'),
    F.avg('Avg_sodium').alias('Avg_sodium'),
    F.avg('Avg_Cholesterol').alias('Avg_Cholesterol'),
    F.avg('Avg_Carbohydrates').alias('Avg_Carbohydrates'),
    F.avg('Avg_Fiber').alias('Avg_Fiber'),
    F.avg('Avg_Sugars').alias('Avg_Sugars'),
    F.avg('Avg_Protein').alias('Avg_Protein'),
    F.avg('Avg_Vitamin_A').alias('Avg_Vitamin_A'),
    F.avg('Avg_Vitamin_C').alias('Avg_Vitamin_C'),
    F.avg('Avg_Calcium').alias('Avg_Calcium'),
    F.avg('Avg_Iron').alias('Avg_Iron')
).collect()

# Calculate the average values for the common columns in food_summary_McD
avg_values_mcd = drink_summary_McD.select(
    F.avg('Avg_Calories').alias('Avg_Calories'),
    F.avg('Avg_T_Fat').alias('Avg_T_Fat'),
    F.avg('Avg_S_Fat').alias('Avg_S_Fat'),
    F.avg('Avg_Ts_Fat').alias('Avg_Ts_Fat'),
    F.avg('Avg_sodium').alias('Avg_sodium'),
    F.avg('Avg_Cholesterol').alias('Avg_Cholesterol'),
    F.avg('Avg_Carbohydrates').alias('Avg_Carbohydrates'),
    F.avg('Avg_Fiber').alias('Avg_Fiber'),
    F.avg('Avg_Sugars').alias('Avg_Sugars'),
    F.avg('Avg_Protein').alias('Avg_Protein'),
    F.avg('Avg_Vitamin_A').alias('Avg_Vitamin_A'),
    F.avg('Avg_Vitamin_C').alias('Avg_Vitamin_C'),
    F.avg('Avg_Calcium').alias('Avg_Calcium'),
    F.avg('Avg_Iron').alias('Avg_Iron')
).collect()

# Extract values from both DataFrames
avg_values_starbucks = avg_values_starbucks[0]
avg_values_mcd = avg_values_mcd[0]

# Prepare the dataset for display
comparison_data = [
    ("Avg_Calories", avg_values_starbucks['Avg_Calories'], avg_values_mcd['Avg_Calories']),
    ("Avg_T_Fat", avg_values_starbucks['Avg_T_Fat'], avg_values_mcd['Avg_T_Fat']),
    ("Avg_S_Fat", avg_values_starbucks['Avg_S_Fat'], avg_values_mcd['Avg_S_Fat']),
    ("Avg_Ts_Fat", avg_values_starbucks['Avg_Ts_Fat'], avg_values_mcd['Avg_Ts_Fat']),
    ("Avg_sodium", avg_values_starbucks['Avg_sodium'], avg_values_mcd['Avg_sodium']),
    ("Avg_Cholesterol", avg_values_starbucks['Avg_Cholesterol'], avg_values_mcd['Avg_Cholesterol']),
    ("Avg_Carbohydrates", avg_values_starbucks['Avg_Carbohydrates'], avg_values_mcd['Avg_Carbohydrates']),
    ("Avg_Fiber", avg_values_starbucks['Avg_Fiber'], avg_values_mcd['Avg_Fiber']),
    ("Avg_Sugars", avg_values_starbucks['Avg_Sugars'], avg_values_mcd['Avg_Sugars']),
    ("Avg_Protein", avg_values_starbucks['Avg_Protein'], avg_values_mcd['Avg_Protein']),
    ("Avg_Vitamin_A", avg_values_starbucks['Avg_Vitamin_A'], avg_values_mcd['Avg_Vitamin_A']),
    ("Avg_Vitamin_C", avg_values_starbucks['Avg_Vitamin_C'], avg_values_mcd['Avg_Vitamin_C']),
    ("Avg_Calcium", avg_values_starbucks['Avg_Calcium'], avg_values_mcd['Avg_Calcium']),
    ("Avg_Iron", avg_values_starbucks['Avg_Iron'], avg_values_mcd['Avg_Iron'])
]

# Convert the comparison data into a DataFrame
comparison_df = spark.createDataFrame(comparison_data, ["Nutritional Factor", "Starbucks Average", "McD Average"])

# Show the dataset
comparison_df.show()

+------------------+-------------------+------------------+
|Nutritional Factor|  Starbucks Average|       McD Average|
+------------------+-------------------+------------------+
|      Avg_Calories| 193.87190082644628|299.46666666666664|
|         Avg_T_Fat|   190.300826446281| 7.733333333333333|
|         Avg_S_Fat|0.03760330578512397|              4.69|
|        Avg_Ts_Fat| 1.3070247933884296|              0.19|
|        Avg_sodium|  6.363636363636363|128.43333333333334|
|   Avg_Cholesterol|  35.99173553719008|25.766666666666666|
| Avg_Carbohydrates| 128.88429752066116|50.266666666666666|
|         Avg_Fiber| 0.8057851239669421|0.7733333333333333|
|        Avg_Sugars|  32.96280991735537|             44.64|
|       Avg_Protein|  6.978512396694215|              7.88|
|     Avg_Vitamin_A|0.09830578512396691|10.433333333333334|
|     Avg_Vitamin_C|0.03648760330578512| 5.526666666666666|
|       Avg_Calcium|0.20756198347107435|25.093333333333334|
|          Avg_Iron|0.07446280991735532|

1. Calories
* Starbucks Average: 193.87
* McDonald's Average: 299.47
* Healthier Option: McDonald's drinks have significantly higher average calories compared to Starbucks drinks, making Starbucks the better choice for calorie-conscious consumers.

2. Total Fat (Avg_T_Fat)
* Starbucks Average: 190.30
* McDonald's Average: 7.73
* Healthier Option: The total fat values for Starbucks appear to be unusually high, suggesting a potential issue in the data (possibly due to incorrect scaling or unit mismatch). This should be reviewed.

3. Saturated Fat (Avg_S_Fat)
* Starbucks Average: 0.038
* McDonald's Average: 4.69
* Healthier Option: McDonald's drinks have a much higher average saturated fat content, which could be less heart-healthy.

4. Trans Fat (Avg_Ts_Fat)
* Starbucks Average: 1.31
* McDonald's Average: 0.19
* Healthier Option: Starbucks drinks contain more trans fat on average, which is concerning due to its adverse health effects.

5. Sodium
* Starbucks Average: 6.36 mg
* McDonald's Average: 128.43 mg
* Healthier Option: McDonald's drinks have extremely high sodium content compared to Starbucks, which may be a concern for individuals monitoring their salt intake.


6. Cholesterol
* Starbucks Average: 35.99 mg
* McDonald's Average: 25.77 mg
* Healthier Option: Starbucks drinks have higher cholesterol levels, which could be a consideration for heart health.

7. Carbohydrates
* Starbucks Average: 128.88 g
* McDonald's Average: 50.27 g
* Healthier Option: Starbucks drinks contain significantly more carbohydrates, likely due to sugar or other carb-heavy ingredients.

8. Fiber
* Starbucks Average: 0.81 g
* McDonald's Average: 0.77 g
* Healthier Option: Both brands have low fiber content, but Starbucks is slightly better in this aspect.

9. Sugars
* Starbucks Average: 32.96 g
* McDonald's Average: 44.64 g
* Healthier Option: McDonald's drinks have a higher average sugar content, which can contribute to weight gain and other health issues.

10. Protein
* Starbucks Average: 6.98 g
* McDonald's Average: 7.88 g
* Healthier Option: McDonald's drinks have slightly higher protein content, which may be beneficial for satiety.

11. Vitamin A
* Starbucks Average: 0.098
* McDonald's Average: 10.43
* Healthier Option: McDonald's drinks provide significantly more Vitamin A, which is beneficial for vision and immunity.

12. Vitamin C
* Starbucks Average: 0.036
* McDonald's Average: 5.53
* Healthier Option: McDonald's drinks are much richer in Vitamin C, which boosts immune health.

13. Calcium
* Starbucks Average: 0.21
* McDonald's Average: 25.09
* Healthier Option: McDonald's drinks provide significantly more calcium, which is crucial for bone health.

14. Iron
* Starbucks Average: 0.074
* McDonald's Average: 2.15
* Healthier Option: McDonald's drinks contain more iron, which supports oxygen transport in the body.




**Overall insights**
* Healthier Option: Starbucks is generally better for calorie, sugar, sodium, and saturated fat-conscious consumers.
* Nutritional Value: McDonald's drinks provide more vitamins (A and C), calcium, and iron, but are less favorable in terms of calories, sodium, and sugar.

# 2. **comparision between starbucks nutrition drink and mcdonalds data**

In [ ]:

drink_summary_McD.printSchema()

root
 |-- Avg_Calories: double (nullable = true)
 |-- Avg_Calories_from_Fat: double (nullable = true)
 |-- Avg_T_Fat: double (nullable = true)
 |-- Avg_T_Fat_DV: double (nullable = true)
 |-- Avg_S_Fat: double (nullable = true)
 |-- Avg_S_Fat_DV: double (nullable = true)
 |-- Avg_Ts_Fat: double (nullable = true)
 |-- Avg_sodium: double (nullable = true)
 |-- Avg_sodium_DV: double (nullable = true)
 |-- Avg_Cholesterol: double (nullable = true)
 |-- Avg_Cholesterol_DV: double (nullable = true)
 |-- Avg_Carbohydrates: double (nullable = true)
 |-- Avg_Carbohydrates_DV: double (nullable = true)
 |-- Avg_Fiber: double (nullable = true)
 |-- Avg_Fiber_DV: double (nullable = true)
 |-- Avg_Sugars: double (nullable = true)
 |-- Avg_Protein: double (nullable = true)
 |-- Avg_Vitamin_A: double (nullable = true)
 |-- Avg_Vitamin_C: double (nullable = true)
 |-- Avg_Calcium: double (nullable = true)
 |-- Avg_Iron: double (nullable = true)



In [ ]:
df_starbucks_drinks_cleaned.printSchema()

root
 |-- Calories: float (nullable = false)
 |-- Fat (g): float (nullable = false)
 |-- Carb_g: float (nullable = false)
 |-- Fiber (g): float (nullable = false)
 |-- Protein: float (nullable = false)
 |-- Sodium: float (nullable = false)



In [ ]:
from pyspark.sql import functions as F

# Calculate the average values for the common columns in df_starbucks_drinks_cleaned & drink_summary_McD

avg_values_star_nutri=df_starbucks_drinks_cleaned.select(
    F.avg('Calories').alias('Avg_Calories'),
    F.avg('Fat (g)').alias('Avg_Fat'),
    F.avg('Carb_g').alias('Avg_Carb_g'),
    F.avg('Fiber (g)').alias('Avg_Fiber'),
    F.avg('Protein').alias('Avg_Protein'),
    F.avg('Sodium').alias('Avg_Sodium')

).collect()

# Calculate the average values for the common columns in food_summary_McD

avg_values_McD = drink_summary_McD.select(
    F.avg('Avg_Calories').alias('Avg_Calories'),
    F.avg('Avg_T_Fat').alias('Avg_Fat'),
    F.avg('Avg_Carbohydrates').alias('Avg_Carb_g'),
    F.avg('Avg_Fiber').alias('Avg_Fiber'),
    F.avg('Avg_Protein').alias('Avg_Protein'),
    F.avg('Avg_sodium').alias('Avg_Sodium')
).collect()


In [ ]:
# Extract values from both DataFrames
avg_values_starbucks_drinks = avg_values_star_nutri[0]
avg_values_mcd = avg_values_McD[0]

# prepare a dataset for display
comparison_data = [
    ("Calories", avg_values_starbucks_drinks['Avg_Calories'], avg_values_mcd['Avg_Calories']),
    ("Fat (g)", avg_values_starbucks_drinks['Avg_Fat'], avg_values_mcd['Avg_Fat']),
    ("Carbohydrates (g)", avg_values_starbucks_drinks['Avg_Carb_g'], avg_values_mcd['Avg_Carb_g']),
    ("Fiber (g)", avg_values_starbucks_drinks['Avg_Fiber'], avg_values_mcd['Avg_Fiber']),
    ("Protein (g)", avg_values_starbucks_drinks['Avg_Protein'], avg_values_mcd['Avg_Protein']),
    ("Sodium (mg)", avg_values_starbucks_drinks['Avg_Sodium'], avg_values_mcd['Avg_Sodium'])
]

In [ ]:
# Convert the comparison data into a DataFrame
comparison_df = spark.createDataFrame(comparison_data, ["Nutritional Factor", "Starbucks Average", "McD Average"])

# Show the dataset
comparison_df.show()

+------------------+-------------------+------------------+
|Nutritional Factor|  Starbucks Average|       McD Average|
+------------------+-------------------+------------------+
|          Calories| 135.16304188528977|299.46666666666664|
|           Fat (g)| 1.2146892655367232| 7.733333333333333|
| Carbohydrates (g)|  24.73913023566122|50.266666666666666|
|         Fiber (g)|0.23163841807909605|0.7733333333333333|
|       Protein (g)|  2.638418079096045|              7.88|
|       Sodium (mg)|  57.93478324588409|128.43333333333334|
+------------------+-------------------+------------------+



1. Calories:
* Starbucks Average: 135.16 kcal
* McDonald's Average: 299.47 kcal
* Healthier Option: On average, McDonald’s items are significantly higher in calories compared to Starbucks. This suggests that Starbucks may be a better choice for those seeking lower-calorie options.
2. Fat (g):
* Starbucks Average: 1.21 g
* McDonald's Average: 7.73 g
* Healthier Option: McDonald’s items contain about 6 times more fat than Starbucks items on average. This indicates that Starbucks offers leaner options, making it preferable for those aiming to reduce fat intake.
3. Carbohydrates (g):
* Starbucks Average: 24.74 g
* McDonald's Average: 50.27 g
* Healthier Option: McDonald’s items have more than double the carbohydrate content of Starbucks. While both brands may have high-carb options, McDonald's meals are typically heavier in carbs.
4. Fiber (g):
* Starbucks Average: 0.23 g
* McDonald's Average: 0.77 g
* Healthier Option: McDonald’s items provide more dietary fiber than Starbucks, though the fiber content in both is relatively low. Neither brand excels in offering high-fiber options.
5. Protein (g):
* Starbucks Average: 2.64 g
* McDonald's Average: 7.88 g
* Healthier Option: McDonald's items are higher in protein content, making them a better choice if protein intake is a priority. Starbucks has lower protein levels, likely due to a focus on beverages and lighter snacks.
6. Sodium (mg):
* Starbucks Average: 57.93 mg
* McDonald's Average: 128.43 mg
* Healthier Option: McDonald's items have over double the sodium content of Starbucks. High sodium intake is associated with health risks like hypertension, so Starbucks offers a more sodium-conscious option.


* **Overall Insights:**
* Health-Conscious Choice: Starbucks is generally better for low-calorie, low-fat, and low-sodium options, making it suitable for lighter or healthier eating.
* Energy and Protein Needs: McDonald's, with higher calories, protein, and carbs, is better for those seeking more substantial meals, such as athletes or individuals with higher energy needs.
